In [1]:
from crypto_volatility_lab.data_construction import CryptoScraper, TimeSeriesCreator
import matplotlib.pyplot as plt
import numpy as np

# Récupération des données

### Récupération d'une seule currency

In [2]:
crypto_scraper = CryptoScraper()
bitcoin_data = crypto_scraper.get_data_for_currency('BTC-USD')

In [ ]:
print(bitcoin_data.shape)
bitcoin_data.head()

#### Création des time series

In [ ]:
ts_creator = TimeSeriesCreator(data=bitcoin_data, date_column_name='Date', value_column_name='Close')
log_returns = ts_creator.create_log_return_time_series()
volatility = ts_creator.create_volatility_time_series()

bitcoin_data["Log Returns"] = log_returns
bitcoin_data["Volatility"] = volatility
bitcoin_data.head(5)

In [ ]:
# sort the data by descending date
bitcoin_data = bitcoin_data.sort_values(by="Date", ascending=True)

# plotting time series by date
fig, ax = plt.subplots(3, 1, figsize=(12, 8))

ax[0].plot(bitcoin_data["Date"], bitcoin_data["Close"])
ax[0].set_title("Bitcoin Price")
ax[0].set_xlabel("Date")
ax[0].set_ylabel("Price")

ax[1].plot(bitcoin_data["Date"], bitcoin_data["Log Returns"])
ax[1].set_title("Log Returns")
ax[1].set_xlabel("Date")
ax[1].set_ylabel("Log Returns")

ax[2].plot(bitcoin_data["Date"], bitcoin_data["Volatility"])
ax[2].set_title("Volatility")
ax[2].set_xlabel("Date")
ax[2].set_ylabel("Volatility")



plt.tight_layout()
plt.show()

In [ ]:
bitcoin_data = bitcoin_data.dropna()

# time split
train_size = int(0.8 * bitcoin_data.shape[0])
train_data = bitcoin_data.iloc[:train_size]
test_data = bitcoin_data.iloc[train_size:]

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(train_data["Date"], train_data["Close"], label="Train")
ax.plot(test_data["Date"], test_data["Close"], label="Test")
start_date = train_data["Date"].min().strftime('%Y-%m-%d')
split_date = test_data["Date"].min().strftime('%Y-%m-%d')
end_date = test_data["Date"].max().strftime('%Y-%m-%d')

ax.set_title(f"Bitcoin Price\n(Start: {start_date}, Split: {split_date}, End: {end_date})")
ax.set_xlabel("Date")
ax.set_ylabel("Price")
ax.legend()
plt.show()


### LSTM

In [ ]:
from crypto_volatility_lab.modeling import LSTMPipeline

pipeline = LSTMPipeline(forecast_horizon=1, epochs=80, batch_size=32, validation_split=0.1)

model = pipeline.fit(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))

In [ ]:
pipeline.evaluate_metrics(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))
pipeline.evaluate_metrics(test_data[["Volatility"]].values, test_data["Volatility"].values.astype(np.float64))

In [ ]:
history = pipeline.get_history()

# plot the training and validation loss
plt.figure(figsize=(12, 8))
plt.plot(history["loss"], label="Training Loss")
plt.plot(history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
# plot y_pred
y_pred = pipeline.predict(train_data[["Volatility"]].values)

# plot the predicted volatility
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(train_data["Date"], train_data["Volatility"], label="True Volatility")
ax.plot(train_data["Date"][-len(y_pred):], y_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")

# predict on test data
y_test_pred = pipeline.predict(test_data[["Volatility"]].values)

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(test_data["Date"], test_data["Volatility"], label="True Volatility")
ax.plot(test_data["Date"][-len(y_test_pred):], y_test_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")

### Modèle GRU

In [ ]:
from crypto_volatility_lab.modeling import GRUPipeline

pipeline = GRUPipeline(forecast_horizon=1, epochs=80, batch_size=32, validation_split=0.1)

model = pipeline.fit(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))

In [ ]:
pipeline.evaluate_metrics(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))
pipeline.evaluate_metrics(test_data[["Volatility"]].values, test_data["Volatility"].values.astype(np.float64))

In [ ]:
history = pipeline.get_history()

# plot the training and validation loss
plt.figure(figsize=(12, 8))
plt.plot(history["loss"], label="Training Loss")
plt.plot(history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
# plot y_pred
y_pred = pipeline.predict(train_data[["Volatility"]].values)

# plot the predicted volatility
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(train_data["Date"], train_data["Volatility"], label="True Volatility")
ax.plot(train_data["Date"][-len(y_pred):], y_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")

# predict on test data
y_test_pred = pipeline.predict(test_data[["Volatility"]].values)

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(test_data["Date"], test_data["Volatility"], label="True Volatility")
ax.plot(test_data["Date"][-len(y_test_pred):], y_test_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")

### Modèle GRU-LSTM

In [ ]:
from crypto_volatility_lab.modeling import LSTMGRUPipeline

pipeline = LSTMGRUPipeline(forecast_horizon=1, epochs=80, batch_size=32, validation_split=0.1)

model = pipeline.fit(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))

In [ ]:
pipeline.evaluate_metrics(train_data[["Volatility"]].values, train_data["Volatility"].values.astype(np.float64))
pipeline.evaluate_metrics(test_data[["Volatility"]].values, test_data["Volatility"].values.astype(np.float64))

In [ ]:
history = pipeline.get_history()

# plot the training and validation loss
plt.figure(figsize=(12, 8))
plt.plot(history["loss"], label="Training Loss")
plt.plot(history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
# plot y_pred
y_pred = pipeline.predict(train_data[["Volatility"]].values)

# plot the predicted volatility
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(train_data["Date"], train_data["Volatility"], label="True Volatility")
ax.plot(train_data["Date"][-len(y_pred):], y_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")

# predict on test data
y_test_pred = pipeline.predict(test_data[["Volatility"]].values)

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(test_data["Date"], test_data["Volatility"], label="True Volatility")
ax.plot(test_data["Date"][-len(y_test_pred):], y_test_pred, label="Predicted Volatility")
ax.set_title("Predicted Volatility")